In [47]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import LogLocator, NullFormatter

In [48]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

def func_fit(x,a,b,c):
    return a+b/(x**2)+c/(x**4)

In [49]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [50]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [51]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [52]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [53]:
def plot_func_multiple_beta_doping(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (28,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 100)
   plt.yticks(fontsize = 100)
   for k in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[k]))
       T = round(1/beta,2)
       plt.errorbar(x_val[:,k],y_val[:,k],yerr = y_val_err[:,k],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))
   plt.grid(True,which='both')
   plt.legend(loc = 'best',ncol = 1,fontsize = 70)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()
    

In [58]:
def plot_func_multiple_U_doping(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(['blue','darkgreen','olive','darkgoldenrod','orange','red'])
   plt.figure(figsize = (25,20))
   #plt.title(plot_title,fontsize = 60)
   #plt.ylim(-0.5,18) 
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 120)
   plt.yticks([0,2,4,6],fontsize = 120)
   plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
   plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
    
   for i in range(len(u_val)):
       c_1 = next(color_1)

       plt.errorbar(x_val[i,:],y_val[i,:],yerr = y_val_err[i,:],c=c_1,marker = 'o',markersize = 20,elinewidth = 3, capsize = 5,label = r"U = %s"%str(u_val[i]))
   #plt.errorbar(x_val_ins,y_val_ins,yerr = y_val_err_ins,c='red',marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"U = 8.0")
   plt.axvline(x=1,color='black',linestyle='dashed',linewidth = 3)
   plt.grid(True,which='both')
   #plt.legend(loc = 'best',ncol=1,fontsize = 80)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()

In [62]:
def equal_time_holon_doublon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,u,Mu,dtau,L):

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Dbn_Dbn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Dbn_Dbn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Den_Dbn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Den_Dbn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Hln_Dbn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Hln_Dbn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
 

   Dbn_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Dbn_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Den_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Den_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Hln_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Hln_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Nrm_Hln_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Nrm_Hln_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))

   G2_Hln_Dbn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   G2_Hln_Dbn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))

    
   Nden = np.zeros((len(u),len(Mu),len(L)))
   Nden_std = np.zeros((len(u),len(Mu),len(L)))
   Ndbn = np.zeros((len(u),len(Mu),len(L)))
   Ndbn_std = np.zeros((len(u),len(Mu),len(L)))
   Nhln = np.zeros((len(u),len(Mu),len(L)))
   Nhln_std = np.zeros((len(u),len(Mu),len(L)))
    
   Hl_db = np.zeros((len(u),len(Mu),len(L)))
   Hl_db_std = np.zeros((len(u),len(Mu),len(L)))

    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
       
   for i in range(len(u)):
       for j in range(len(Mu)):
           for k in range(len(L)):
                       
               Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,Mu[j],dtau,L[k])

               filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],Mu[j],dtau,L[k])
               with open(filename_eqm_avg, 'rb') as infile:
                    sys_measure_avg = pickle.load(infile)

               filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],Mu[j],dtau,L[k])
               with open(filename_eqm_std, 'rb') as infile:
                    sys_measure_std = pickle.load(infile)

               Nden[i][j][k] = sys_measure_avg['Density averaged']
               Nden_std[i][j][k] = sys_measure_std['Density standard deviation']
               Ndbn[i][j][k] = sys_measure_avg['Doublon number averaged']
               Ndbn_std[i][j][k] = sys_measure_std['Doublon number standard deviation']
               Nhln[i][j][k] = np.abs(1-Nden[i][j][k]+Ndbn[i][j][k])
               Nhln_std[i][j][k] = np.abs(np.sqrt(Ndbn_std[i][j][k]*Ndbn_std[i][j][k]+Nden_std[i][j][k]*Nden_std[i][j][k]))
               Hl_db[i][j][k] = Ndbn[i][j][k]*Nhln[i][j][k]
               Hl_db_std[i][j][k] = Hl_db[i][j][k]*(Ndbn_std[i][j][k]/(Ndbn[i][j][k]+1e-6)+Nhln_std[i][j][k]/(Nhln[i][j][k]+1e-6))
           
               Text_dir_dbn_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
               Text_dir_den_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
               Text_dir_dbn_dbn_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
               Text_dir_den_dbn_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])

               filename_dbn_dbn_corr = '%s/Equal_time_Doublon_doublon_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_corr,N,u[i],Mu[j],dtau,L[k])
               Dbn_Dbn_corr[i,j,k,:],Dbn_Dbn_corr_std[i,j,k,:] = np.loadtxt(filename_dbn_dbn_corr,unpack = 'True', usecols = [1,2])

               filename_den_dbn_corr = '%s/Equal_time_Density_Doublon_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_corr,N,u[i],Mu[j],dtau,L[k])
               Den_Dbn_corr[i,j,k,:],Den_Dbn_corr_std[i,j,k,:] = np.loadtxt(filename_den_dbn_corr,unpack = 'True', usecols = [1,2])

               filename_dbn_dbn_conn_corr = '%s/Equal_time_Doublon_doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_conn_corr,N,u[i],Mu[j],dtau,L[k])
               Dbn_Dbn_conn_corr[i,j,k,:],Dbn_Dbn_conn_corr_std[i,j,k,:] = np.loadtxt(filename_dbn_dbn_conn_corr,unpack = 'True', usecols = [1,2])

               filename_den_dbn_conn_corr = '%s/Equal_time_Density_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_conn_corr,N,u[i],Mu[j],dtau,L[k])
               Den_Dbn_conn_corr[i,j,k,:],Den_Dbn_conn_corr_std[i,j,k,:] = np.loadtxt(filename_den_dbn_conn_corr,unpack = 'True', usecols = [1,2])

               Hln_Dbn_corr[i,j,k,:] = np.add(np.subtract(Dbn_Dbn_corr[i,j,k,:],Den_Dbn_corr[i,j,k,:]),Ndbn[i][j][k]*np.ones(len(Den_Dbn_corr[i,j,k,:])))
               Hln_Dbn_corr_std[i,j,k,:] = np.sqrt(np.add(np.add(np.power(Dbn_Dbn_corr_std[i,j,k,:],2),np.power(Den_Dbn_corr_std[i,j,k,:],2)),Ndbn_std[i][j][k]*Ndbn_std[i][j][k]*Ndbn[i][j][k]*np.ones(len(Den_Dbn_conn_corr[i,j,k,:]))))
           
               #Hln_Dbn_conn_corr[i,j,k,:] = np.subtract(Dbn_Dbn_conn_corr[i,j,k,:],Den_Dbn_conn_corr[i,j,k,:])
               #Hln_Dbn_conn_corr_std[i,j,k,:] = np.sqrt(np.add(np.power(Dbn_Dbn_conn_corr_std[i,j,k,:],2),np.power(Den_Dbn_conn_corr_std[i,j,k,:],2)))

   for i in range(len(u)):
       for j in range(int(len(Mu)/2)+1):
           for k in range(len(L)):
               for r in range(len(r_2_unq)):
                   
                   Hln_Dbn_conn_corr[i,j,k,:] = np.subtract(Dbn_Dbn_conn_corr[i,j,k,:],Den_Dbn_conn_corr[i,j,k,:])
                   Hln_Dbn_conn_corr_std[i,j,k,:] = np.sqrt(np.add(np.power(Dbn_Dbn_conn_corr_std[i,j,k,:],2),np.power(Den_Dbn_conn_corr_std[i,j,k,:],2)))
                   Hln_Dbn_conn_corr[i,len(Mu)-j-1,k,r] = Hln_Dbn_conn_corr[i,j,k,r]
                   Hln_Dbn_conn_corr_std[i,len(Mu)-j-1,k,r] = Hln_Dbn_conn_corr_std[i,j,k,r]
                   
                   Nrm_Hln_Dbn_conn_corr[i,j,k,r] = Hln_Dbn_conn_corr[i,j,k,r]/(Hl_db[i,j,k]+1e-6)
                   Nrm_Hln_Dbn_conn_corr[i,len(Mu)-j-1,k,r] = Hln_Dbn_conn_corr[i,j,k,r]/(Hl_db[i,j,k]+1e-6)
                   Nrm_Hln_Dbn_conn_corr_std[i,j,k,r] = np.abs(Nrm_Hln_Dbn_conn_corr[i,j,k,r]*((Hln_Dbn_conn_corr_std[i,j,k,r]+1e-8)/(Hln_Dbn_conn_corr[i,j,k,r]+1e-8))) #+(Hl_db_std[i][j][k]+1e-8)/(Hl_db[i][j][k]+1e-8)))
                   Nrm_Hln_Dbn_conn_corr_std[i,len(Mu)-j-1,k,r] = np.abs(Nrm_Hln_Dbn_conn_corr[i,j,k,r]*((Hln_Dbn_conn_corr_std[i,j,k,r]+1e-8)/(Hln_Dbn_conn_corr[i,j,k,r]+1e-8))) #+(Hl_db_std[i][j][k]+1e-8)/(Hl_db[i][j][k]+1e-8)))
   #print("hl_db",Hl_db)
                   G2_Hln_Dbn_corr[i,j,k,r] = Hln_Dbn_corr[i,j,k,r]/(Hl_db[i,j,k]+1e-6)
                   G2_Hln_Dbn_corr[i,len(Mu)-j-1,k,r] = Hln_Dbn_corr[i,j,k,r]/(Hl_db[i,j,k]+1e-6)
                   G2_Hln_Dbn_corr_std[i,j,k,r] = np.abs(Nrm_Hln_Dbn_conn_corr[i,j,k,r]*((Hln_Dbn_conn_corr_std[i,j,k,r]+1e-8)/(Hln_Dbn_conn_corr[i,j,k,r]+1e-8))) #+(Hl_db_std[i][j][k]+1e-8)/(Hl_db[i][j][k]+1e-8)))
                   G2_Hln_Dbn_corr_std[i,len(Mu)-j-1,k,r] = np.abs(Nrm_Hln_Dbn_conn_corr[i,j,k,r]*((Hln_Dbn_conn_corr_std[i,j,k,r]+1e-8)/(Hln_Dbn_conn_corr[i,j,k,r]+1e-8))) #+(Hl_db_std[i][j][k]+1e-8)/(Hl_db[i][j][k]+1e-8)))
    
    
    
   print("Nrm_dbn_std",Nrm_Hln_Dbn_conn_corr_std[:,:,:,1]) 
   Hln_Dbn_corr_n0 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n1 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n2 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n3 = np.zeros((len(u),len(Mu),len(L)))

   Hln_Dbn_corr_n0_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n1_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n2_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_corr_n3_std = np.zeros((len(u),len(Mu),len(L)))

   Hln_Dbn_conn_corr_n0 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n1 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n2 = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n3 = np.zeros((len(u),len(Mu),len(L)))
    
   Hln_Dbn_conn_corr_n0_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n1_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n2_std = np.zeros((len(u),len(Mu),len(L)))
   Hln_Dbn_conn_corr_n3_std = np.zeros((len(u),len(Mu),len(L)))

   Nrm_Hln_Dbn_conn_corr_n0 = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n1 = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n2 = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n3 = np.zeros((len(u),len(Mu),len(L)))
    
   Nrm_Hln_Dbn_conn_corr_n0_std = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n1_std = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n2_std = np.zeros((len(u),len(Mu),len(L)))
   Nrm_Hln_Dbn_conn_corr_n3_std = np.zeros((len(u),len(Mu),len(L)))


   for i in range(len(u)):
       for j in range(len(Mu)):
           for k in range(len(L)):

               Hln_Dbn_corr_n0[i][j][k] = Hln_Dbn_corr[i][j][k][0]
               Hln_Dbn_corr_n1[i][j][k] = Hln_Dbn_corr[i][j][k][1]
               Hln_Dbn_corr_n2[i][j][k] = Hln_Dbn_corr[i][j][k][2]
               Hln_Dbn_corr_n3[i][j][k] = Hln_Dbn_corr[i][j][k][3]

               Hln_Dbn_corr_n0_std[i][j][k] = Hln_Dbn_corr_std[i][j][k][0]
               Hln_Dbn_corr_n1_std[i][j][k] = Hln_Dbn_corr_std[i][j][k][1]
               Hln_Dbn_corr_n2_std[i][j][k] = Hln_Dbn_corr_std[i][j][k][2]
               Hln_Dbn_corr_n3_std[i][j][k] = Hln_Dbn_corr_std[i][j][k][3]

               Hln_Dbn_conn_corr_n0[i][j][k] = Hln_Dbn_conn_corr[i][j][k][0]
               Hln_Dbn_conn_corr_n1[i][j][k] = Hln_Dbn_conn_corr[i][j][k][1]
               Hln_Dbn_conn_corr_n2[i][j][k] = Hln_Dbn_conn_corr[i][j][k][2]
               Hln_Dbn_conn_corr_n3[i][j][k] = Hln_Dbn_conn_corr[i][j][k][3]
           
               Hln_Dbn_conn_corr_n0_std[i][j][k] = Hln_Dbn_conn_corr_std[i][j][k][0]
               Hln_Dbn_conn_corr_n1_std[i][j][k] = Hln_Dbn_conn_corr_std[i][j][k][1]
               Hln_Dbn_conn_corr_n2_std[i][j][k] = Hln_Dbn_conn_corr_std[i][j][k][2]
               Hln_Dbn_conn_corr_n3_std[i][j][k] = Hln_Dbn_conn_corr_std[i][j][k][3]

               Nrm_Hln_Dbn_conn_corr_n0[i][j][k] = Nrm_Hln_Dbn_conn_corr[i][j][k][0]
               Nrm_Hln_Dbn_conn_corr_n1[i][j][k] = Nrm_Hln_Dbn_conn_corr[i][j][k][1]
               Nrm_Hln_Dbn_conn_corr_n2[i][j][k] = Nrm_Hln_Dbn_conn_corr[i][j][k][2]
               Nrm_Hln_Dbn_conn_corr_n3[i][j][k] = Nrm_Hln_Dbn_conn_corr[i][j][k][3]
           
               Nrm_Hln_Dbn_conn_corr_n0_std[i][j][k] = Nrm_Hln_Dbn_conn_corr_std[i][j][k][0]
               Nrm_Hln_Dbn_conn_corr_n1_std[i][j][k] = Nrm_Hln_Dbn_conn_corr_std[i][j][k][1]
               Nrm_Hln_Dbn_conn_corr_n2_std[i][j][k] = Nrm_Hln_Dbn_conn_corr_std[i][j][k][2]
               Nrm_Hln_Dbn_conn_corr_n3_std[i][j][k] = Nrm_Hln_Dbn_conn_corr_std[i][j][k][3]
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"

   for i in range(len(u)):
       
       Pl_name_n0 = "%s/Holon_doublon_correlation_onsite_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       Pl_name_conn_n0 = "%s/Holon_doublon_correlation_connected_onsite_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       Pl_name_nrm_conn_n0 = "%s/Normalized_Holon_doublon_correlation_connected_onsite_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)

       Pl_name_n1 = "%s/Holon_doublon_correlation_1st_nearest_neighbor_averaged_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       Pl_name_conn_n1 = "%s/Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       Pl_name_conn_n2 = "%s/Holon_doublon_correlation_connected_2nd_nearest_neighbor_averaged_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       
       Pl_name_nrm_conn_n1 = "%s/Normalized_Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)

       #Pl_name_hl_db = "%s/Holon_doublon_product_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Hln_Dbn_corr_n0[i,:,:],Hln_Dbn_corr_n1_std[i,:,:],Pl_name_n0)
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Hln_Dbn_conn_corr_n0[i,:,:],Hln_Dbn_conn_corr_n1_std[i,:,:],Pl_name_conn_n0)
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Nrm_Hln_Dbn_conn_corr_n0[i,:,:],Nrm_Hln_Dbn_conn_corr_n1_std[i,:,:],Pl_name_nrm_conn_n0)
   
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Hln_Dbn_corr_n1[i,:,:],Hln_Dbn_corr_n1_std[i,:,:],Pl_name_n1)
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Hln_Dbn_conn_corr_n1[i,:,:],Hln_Dbn_conn_corr_n1_std[i,:,:],Pl_name_conn_n1)
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Nrm_Hln_Dbn_conn_corr_n1[i,:,:],Nrm_Hln_Dbn_conn_corr_n1_std[i,:,:],Pl_name_nrm_conn_n1)
       #plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Hl_db[i,:,:],Hl_db_std[i,:,:],Pl_name_hl_db)
       
   for k in range(len(L)):
       
       Pl_name_n0 = "%s/Holon_doublon_correlation_onsite_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_conn_n0 = "%s/Holon_doublon_correlation_connected_onsite_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_nrm_conn_n0 = "%s/Normalized_Holon_doublon_correlation_connected_onsite_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])

       Pl_name_n1 = "%s/Holon_doublon_correlation_1st_nearest_neighbor_averaged_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_conn_n1 = "%s/Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_conn_n2 = "%s/Holon_doublon_correlation_connected_2nd_nearest_neighbor_averaged_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_nrm_conn_n1 = "%s/Normalized_Holon_doublon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])

       Pl_name_hl_db = "%s/Holon_doublon_product_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hln_Dbn_corr_n0[:,:,k],Hln_Dbn_corr_n1_std[:,:,k],Pl_name_n0)
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hln_Dbn_conn_corr_n0[:,:,k],Hln_Dbn_conn_corr_n1_std[:,:,k],Pl_name_conn_n0)
       plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n3,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Nrm_Hln_Dbn_conn_corr_n0[:,:,k],Nrm_Hln_Dbn_conn_corr_n1_std[:,:,k],Pl_name_nrm_conn_n0)
   
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hln_Dbn_corr_n1[:,:,k],Hln_Dbn_corr_n1_std[:,:,k],Pl_name_n1)
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hln_Dbn_conn_corr_n1[:,:,k],Hln_Dbn_conn_corr_n1_std[:,:,k],Pl_name_conn_n1)
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hln_Dbn_conn_corr_n2[:,:,k],Hln_Dbn_conn_corr_n2_std[:,:,k],Pl_name_conn_n2)
       
       plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n3,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Nrm_Hln_Dbn_conn_corr_n1[:,:,k],Nrm_Hln_Dbn_conn_corr_n1_std[:,:,k],Pl_name_nrm_conn_n1)
       #plot_func_multiple_U_doping(N,u,Mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Hl_db[:,:,k],Hl_db_std[:,:,k],Pl_name_hl_db)
       



In [63]:
def main():

    N = "10"
    Dtau = "0.05"

    #U_hf=["1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00","7.50","8.00","8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00"] #,"8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00","12.50"]
    U = ["1.0","4.0","5.0","6.0","8.0","10.0"]
    Mu = ["-10.00","-9.50","-9.00","-8.50","-8.00","-7.50","-7.00","-6.50","-6.00","-5.50","-5.00","-4.50","-4.00","-3.50","-3.00","-2.50",
          "-2.00","-1.50","-1.00","-0.50","0.00","0.50","1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00",
          "7.50","8.00","8.50","9.00","9.50","10.00"]
    Mu_hf ="0.00"
    Trot = ["30"] #,"30","40","50","60"]

    
    Text_dir_main_corr = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations'%N
    Text_dir_main_eqm = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s'%N
    Text_dir_main_corr_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations_half_filling'%N
    Text_dir_main_eqm_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_half_filling'%N
    
    

    Graph_dir = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Doublon_holon_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir):
        os.makedirs(Graph_dir)
        
    #equal_time_holon_doublon_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm_hf,Graph_dir_hf,N,U_hf,Mu_hf,Dtau,Trot)
    equal_time_holon_doublon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)


In [64]:
main()

Nrm_dbn_std [[[0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [1.07734517e-03]
  [1.24140613e-03]
  [4.01839896e-04]
  [1.12693541e-04]
  [3.71776779e-05]
  [2.61961092e-05]
  [2.42409305e-05]
  [3.46191825e-05]
  [4.38359219e-05]
  [5.58208053e-05]
  [8.36097858e-05]
  [1.13760691e-04]
  [1.77629432e-04]
  [2.42918624e-04]
  [4.57165096e-04]
  [2.42918624e-04]
  [1.77629432e-04]
  [1.13760691e-04]
  [8.36097858e-05]
  [5.58208053e-05]
  [4.38359219e-05]
  [3.46191825e-05]
  [2.42409305e-05]
  [2.61961092e-05]
  [3.71776779e-05]
  [1.12693541e-04]
  [4.01839896e-04]
  [1.24140613e-03]
  [1.07734517e-03]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]]

 [[0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [1.78223620e-03]
  [9.07707709e-04]
  [2.89283988e-04]
  [1.02230208e-04]
  [6.70739115e-05]
  [6.84210657e-05]
  [7.77412123e-05